In [2]:
import numpy as np
import pandas as pd
import os
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import normalize
datasetFolder = "isc_datasets"
def loadData_txt(filename,delimiter):
    filePath = os.path.join(datasetFolder,filename + ".txt")
    return pd.read_table(filePath,delimiter = delimiter)
dataset = loadData_txt("transfusionData",',')
dataset.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [3]:
transfusionData = dataset.to_numpy()
transfusionData = np.delete(transfusionData,-1,1)
print(transfusionData)


[[    2    50 12500    98]
 [    0    13  3250    28]
 [    1    16  4000    35]
 ...
 [   23     3   750    62]
 [   39     1   250    39]
 [   72     1   250    72]]


In [4]:
#returns -> [[clusters in a 1D subspace -> each cluster [no of datapoint in a cluster] ] , [clusters in a 1D subspac] , ...] and an array that indicates uncovered elements in subspace clustering process
def find1DSubspaceClusters(eps,minpts,dataSet):
    """
    clusters 1D subspaces of the data 

    uncovered elements are in [[[dims of clustering],number of uncovered elements],[],... ]
    
    Return : [[clusters in a 1D subspace -> each cluster [no of datapoint in a cluster] ] , [clusters in a 1D subspac] , ...] , an array that indicates uncovered elements in subspace clustering process
    """
    
    one_D_subspaces = dataSet.T
    one_D_clusters = []
    result = []
    #clustering proccess
    for i in one_D_subspaces:
        i = i.reshape(-1,1)
        clustering = DBSCAN(eps = eps,min_samples = minpts).fit(i)
        one_D_clusters.append(clustering)
    #turning labels_ into ndarray of clusters 
    #we dont want the noisy samples so we're gone skip em
    uncovered_iterator = 0
    for i in one_D_clusters:
        points = i.labels_
        LabelsNum = max(points) + 1
        esets=[[] for i in range(LabelsNum)]
        uncoveredLen = [[[i],0] for i in range(len(one_D_clusters))]
        for j in range(len(points)):
            if points[j] != -1 :
                esets[points[j]].append(j)
            else:
                uncoveredLen[uncovered_iterator][1] += 1
        uncovered_iterator +=1        
        if(len(esets) != 0):
            result.append(esets)
    return result,uncoveredLen          

clusters,uncovered = find1DSubspaceClusters(0.2,6,transfusionData)     
print("no of 1D subspaces :",len(clusters),"\n")
print("1D subspaces and their clusters :","\n")
for i in clusters:
    print("no of clusters: ",len(i),"\n")
    for j in i:
        print(j,"\n")
print(uncovered)


no of 1D subspaces : 4 

1D subspaces and their clusters : 

no of clusters:  14 

[0, 3, 6, 8, 12, 14, 15, 16, 17, 18, 19, 20, 22, 23, 34, 35, 36, 38, 40, 41, 42, 43, 46, 47, 48, 49, 50, 51, 52, 53, 57, 58, 60, 61, 62, 64, 66, 70, 71, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 84, 85, 89, 101, 103, 104, 108, 109, 114, 116, 119, 120, 122, 123, 124, 125, 127, 129, 131, 132, 134, 135, 136, 137, 139, 140, 147, 148, 149, 150, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 179, 182, 192, 194, 195, 206, 207, 238, 251, 255, 262, 263, 295, 303, 326, 500, 502, 503, 505, 507, 508, 509, 511, 512, 520, 521, 525, 526, 527, 529, 531, 533, 534, 536, 537, 538, 539, 540, 541, 547, 548, 551, 553, 554, 565, 569, 570, 572, 573, 574, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 592, 598, 600, 606, 607, 628, 631, 647, 669, 670] 

[2, 4, 7, 13, 30, 69, 118, 523, 544] 

[5, 10, 21, 25, 26, 27, 28, 29, 31, 32, 33, 37, 39, 44, 45, 54, 55

In [5]:
from copy import copy
from pandas import array
from scikit_roughsets.roughsets import RoughSetsReducer
reducer = RoughSetsReducer()

def findLowApprx(P,Q,i,U):
    """
    P and Q are subspaces

    this method finds P-lower Approximation of Qi(ith cluster in Q subspace)

    P has to be a np.array in which the dimensions indexes needed to use are stored(indexes > 0) 
    """
    
    return reducer.rslower(Q[i],P,U)


def attrDependency(P,Q,U):
    """
    calculates Attribute dependency measure between P and Q subspaces

    P has to be a np.array in which the dimensions indexes needed to use are stored(indexes > 0) 
    
    """
    
    AllOfLowApprxs = 0
    for i in range(len(Q)):
        AllOfLowApprxs += len(findLowApprx(P,Q,i,U))
    return AllOfLowApprxs / len(U)

        
def isInterestingSubset(P,Q,U,gamma):
    """P has to be a np.array in which the dimensions indexes needed to use are stored(indexes > 0) """
    
    return bool(attrDependency(P,Q,U) < gamma)


def findBestSubspace(eligibleSubspace,uncoveredLen):
    """
     returns list of dimensions that are best subspace together

     uncoveredlen -> [[[dims of clustering],number of uncovered elements],[],... ]

     for example eligible subspace would be 2D and we're looking for the best 1D subspace

     eligible subspace is [p,q] whereas p and q are subspaces that are interesting subset together
    
     also works for higher dims
    """
    print("finding best subspace for :",eligibleSubspace)
    uncovered_eligibleSubspacesOnly = []
    print(uncoveredLen)
    for i in range(len(uncoveredLen)):
        subspacefinder = np.isin(np.setdiff1d(eligibleSubspace,uncoveredLen[i][0]),eligibleSubspace)
        print(" i : ",uncoveredLen[i]," subspace finder array : ",subspacefinder)
        if all(subspacefinder) and len(subspacefinder) == 1:
            uncovered_eligibleSubspacesOnly.append(uncoveredLen[i])
    print(uncovered_eligibleSubspacesOnly)
    bestSubspaceIndex = uncovered_eligibleSubspacesOnly.index(max(uncovered_eligibleSubspacesOnly))
    return [i for i in eligibleSubspace if i != bestSubspaceIndex]


def getClusteredpointsfromDims(Dims,clustered_Subspaces):
    """
    this method finds the >2 dimensional subspace's points clustered based on dims given
    """
    npclstrs_T =np.array(clustered_Subspaces[len(Dims) - 2]).T
    for dim_ix in range(len(npclstrs_T[0])):
        if npclstrs_T[0][dim_ix] == Dims:
            indexJ = dim_ix
    bestSubspaceChunk = clustered_Subspaces[len(Dims)-2][indexJ][1]
    return bestSubspaceChunk 

def getSubspace(dimList,data):
    '''   
    returns 2D subspace made of two 1D subspaces p and q

    Should also work for other dimensions

    data has to be numpy array in (records,dimensions) shape

    dimList in the [p,q] in which p and q are the # of dimensions we're working on
    '''
    return data[:,dimList]


def clstrToPointsList(clstrObj ,clstrPointsList):
    """ 
    clstrObj is the result of clustering 

    clstrPointsList is the cluster in which the number of data points included in the cluster are stored
    
    returns ndarray of point numbers in their cluster after the clustering result based on DBSCAN method and an array that indicates the uncovered elements in the clustering process
    """
    labels = clstrObj.labels_
    labelMx = max(labels) + 1
    clstrs = [[] for i in range(labelMx)]
    uncoveredLen = 0
    for i in range(len(labels)):
        if labels[i] != -1:
            clstrs[labels[i]].append(clstrPointsList[i])
        else:
            uncoveredLen += 1
    return clstrs,uncoveredLen


def partitionBestSubspace(eps,minpts,bestSubspaceClustered,dimList,data):
    """ 
    This will partition the clusters in best subspace using DBSCAN based on distances in the 2D subspace(Or the higher one)

    returns the result of clustering in  ND array of number of the points, uncovered elements

    Input params:
    ---
    Best subspace is 1D and eligible subspace is 2D

    Eligible subspace is [p,q] whereas p and q are subspaces that are interesting subset together

    Data has to be numpy array in (records,dimensions) shape

    DimList in the [p,q] in which p and q are the # of dimensions we're working on
    
    """
    rawEligible2dSubspace = getSubspace(dimList,data)
    uncovered_total = 0
    result = []
    for clstr in bestSubspaceClustered:
        clstrInHighD = np.array([rawEligible2dSubspace[i] for i in clstr])
        rsltForClstr  = DBSCAN(eps = eps, min_samples = minpts).fit(clstrInHighD)
        rsltForClstr_in_points,uncovered = clstrToPointsList(rsltForClstr,clstr)
        result.append(rsltForClstr_in_points)
        uncovered_total += uncovered
    return result,uncovered_total


def findEligibleSubspaces(eligible_Subspaces,one_D_clusters,current_dim,gamma,Data):
    """
    finds eligible subspaces 
    ---
    used for ND > 1, 1D to 2D is handeled in ISC()

    Return:  [[i,j,k,..],[]] , i j k are # of dimensions
    """
    
    atLeastOneHD = False
    for ND_eligible_index in range(len(eligible_Subspaces)):
        for attr_j in range(len(one_D_clusters)):
            if len(eligible_Subspaces[ND_eligible_index]) == current_dim and  \
                attr_j not in eligible_Subspaces[ND_eligible_index] and \
                isInterestingSubset(np.array([i+1 for i in eligible_Subspaces[ND_eligible_index]]),one_D_clusters[attr_j],Data,gamma):
                eligible_Subspaces.append(eligible_Subspaces[ND_eligible_index] + [attr_j])
                atLeastOneHD = True
    return eligible_Subspaces,atLeastOneHD


def partitionEligibleSubspaces(eligible_Subspaces,one_D_clusters,uncovered_LD,clustered_Subspaces,current_dim,eps,minPts,Data):
    """
    eligible_subspaces = [[i,j,k],...]: i,j,k are # of dims
    
    Return: clustered_Subspaces are the result we want : [[[subspaces],[clusters],[uncoveredLen]],...]
    """
    uncovered_total = []
    for eligible_Subspace in eligible_Subspaces:
        if len(eligible_Subspace) == current_dim:
            bestSubspace_dims = findBestSubspace([one_D_clusters[i] for i in eligible_Subspace],uncovered_LD)
            HD_ClusteredPoints,uncovered = partitionBestSubspace(eps,minPts,getClusteredpointsfromDims(bestSubspace_dims,clustered_Subspaces),eligible_Subspace,Data)
            clustered_Subspaces[current_dim-2].append([eligible_Subspace,HD_ClusteredPoints,uncovered])
            uncovered_total.append(uncovered)
    return clustered_Subspaces

def DimSubspace_isEmpty(dimNum,clustered_Subspaces):
    AllDimClusters = clustered_Subspaces[dimNum-2]
    return all(not i[1] for i in AllDimClusters)


def denseCluster_found(clusters,threshold,dim):
    """used as one of the base cases for clustering more than 1D->2D dimensions
    
    input params:
    ---
    Clusters -> [[for 2D clusters],[for 3D clusters],...]: each [] for clusters in the specific dimension in which -> [[subspaces],[clusters],[uncoveredElements]] : we'll call them clusterChunks

    threshold: int : if the number of points in a cluster is less it is Not considered Dense

    dim: int > 1 : in which dimension we're looking for dense clusters
    
    Return: return_description
    """
    
    pass

one_D_clusters,uncovered_1d = find1DSubspaceClusters(1,5,transfusionData)
p = one_D_clusters[2]
q = one_D_clusters[0]
i = 1
db_size = len(transfusionData)
#print(findLowApprx(np.array([2]),q,i,transfusionData))
#print(attrDependency(np.array([2]),q,transfusionData))
print(findBestSubspace([2,0],uncovered_1d))
#partitionBestSubspace(2,4,p,[0,2],transfusionData)

finding best subspace for : [2, 0]
[[[0], 0], [[1], 0], [[2], 0], [[3], 7]]
 i :  [[0], 0]  subspace finder array :  [ True]
 i :  [[1], 0]  subspace finder array :  [ True  True]
 i :  [[2], 0]  subspace finder array :  [ True]
 i :  [[3], 7]  subspace finder array :  [ True  True]
[[[0], 0], [[2], 0]]
[2, 0]


In [8]:
#Time for the Algorithm itself
from numpy import ndarray
def ISC(eps: float,minPts : float,gamma : float,Data : ndarray):
    """clusteres the data given in the numpy ndarray format based on ISC clustering algorithm
    
    eps and minpts are used in DBSCAN and gamma (0,1) is the attribute dependency threshold
    Return: 
    """
    
    uncoveredElementsList = []
    one_D_clusters,uncovered_1D = find1DSubspaceClusters(eps,minPts,Data)
    db_size = len(Data)
    eligible_Subspaces = []
    # in clustered_Subspaces we're gone store High dimensional (>1) subspaces that we just clustered
    # the first (in future) element would be the 2D clustering result based on ISC algorithm
    # e.g in the 2D clstr part we have to specify the dims so the elements
    clustered_Subspaces= [[] for i in range(len(one_D_clusters) - 1)]
    
    for attr_i in range(len(one_D_clusters)):
        for attr_j in range(attr_i + 1 ,len(one_D_clusters)):
            if isInterestingSubset(np.array([attr_i+1]),one_D_clusters[attr_j],Data,gamma):
                eligible_Subspaces.append([attr_i,attr_j])
    print("eligible 2Ds : ",eligible_Subspaces)
    #1D to 2D process:
    uncovered_2D = []
    for eligible_Subspace in eligible_Subspaces:
        bestSubspace = findBestSubspace([i for i in eligible_Subspace],uncovered_1D)
        HD_ClusteredPoints,uncovered_2D_temp = partitionBestSubspace(eps,minPts,one_D_clusters[bestSubspace[0]],eligible_Subspace,Data)
        clustered_Subspaces[0].append([eligible_Subspace,HD_ClusteredPoints])
        uncovered_2D.append([eligible_Subspace,uncovered_2D_temp])
    #finding eligible subspaces in the N+1 Dimensional space
    #then we're gone partition the best subspace of each group 
    #we'll continue this process untill no more subspaces can be formed
    current_dim = 2
    uncovered_mem = [i for  i in uncovered_2D]
    while True:
        eligible_Subspaces,atLeastOneHD = findEligibleSubspaces(eligible_Subspaces, one_D_clusters,current_dim,gamma,Data)
        print(eligible_Subspaces)
        if not atLeastOneHD: 
            break
        else:
            current_dim+=1 
            clustered_Subspaces,uncovered_mem = partitionEligibleSubspaces(eligible_Subspaces,one_D_clusters,uncovered_mem,clustered_Subspaces,current_dim,eps,minPts,Data)
    return clustered_Subspaces
clustered = ISC(1,2,0.6,transfusionData)
clustered = np.array(clustered)
print(clustered)

eligible 2Ds :  [[0, 2], [0, 3], [1, 2], [1, 3], [2, 3]]
finding best subspace for : [0, 2]
[[[0], 0], [[1], 0], [[2], 0], [[3], 0]]
 i :  [[0], 0]  subspace finder array :  [ True]
 i :  [[1], 0]  subspace finder array :  [ True  True]
 i :  [[2], 0]  subspace finder array :  [ True]
 i :  [[3], 0]  subspace finder array :  [ True  True]
[[[0], 0], [[2], 0]]
finding best subspace for : [0, 3]
[[[0], 0], [[1], 0], [[2], 0], [[3], 0]]
 i :  [[0], 0]  subspace finder array :  [ True]
 i :  [[1], 0]  subspace finder array :  [ True  True]
 i :  [[2], 0]  subspace finder array :  [ True  True]
 i :  [[3], 0]  subspace finder array :  [ True]
[[[0], 0], [[3], 0]]
finding best subspace for : [1, 2]
[[[0], 0], [[1], 0], [[2], 0], [[3], 0]]
 i :  [[0], 0]  subspace finder array :  [ True  True]
 i :  [[1], 0]  subspace finder array :  [ True]
 i :  [[2], 0]  subspace finder array :  [ True]
 i :  [[3], 0]  subspace finder array :  [ True  True]
[[[1], 0], [[2], 0]]
finding best subspace for : 

c:\Users\TAHA\HandsOnMachineLearningEnvironment\env\lib\site-packages\numpy\lib\arraysetops.py:272: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)
c:\Users\TAHA\HandsOnMachineLearningEnvironment\env\lib\site-packages\numpy\lib\arraysetops.py:591: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar2 = np.asarray(ar2).ravel()


ValueError: max() arg is an empty sequence